In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings("ignore")
# Setting the parameter to plot at 30x15 size
plt.rcParams['figure.figsize'] = 30, 15

In [5]:
# Declare the locations of the files
##root_folder_train = 'C:\\Users\\animi\\Documents\\Dataset\\ECG Dataset\\Train\\'
root_folder_train = 'D:\pythonWS\\TMP\\ECGDATA\\train\\'
##root_folder_test = 'C:\\Users\\animi\\Documents\\Dataset\\ECG Dataset\\Test\\'
root_folder_test = 'D:\\pythonWS\\TMP\\ECGDATA\\test\\'
category = ['Normal', 'Abnormal']
##destination = 'C:\\Users\\animi\\Documents\\Dataset\\ECG Dataset\\Customized_Data\\'
destination = 'D:\\pythonWS\\TMP\\ECGDATA\\Customized_Data\\'

In [6]:
# Function to load the dataset and do the required formatting to be able to feed the data into the Keras Model
def load_data(root_folder):
    # These are delcarations of variables that have been used inside the for loop
    final_list = list()
    labels = list()
    # It is iterating through both the categories Normal and Abnormal
    for cat in category:
        # It is taking and processing each file in the folder
        for filename in os.listdir(root_folder+cat):
            
            # Read each file for each category and drop the unnecessary columns
            path = root_folder + cat + '\\' + filename
            df = pd.read_csv(path) # Read the CSV using inbuilt Pandas Function
            df.drop(index=0, axis=0, inplace=True) # Drop the first row, which contains the units of measurement (useless for our use case)
            df.columns = ["time", "ECG0", "ECG1"] # Rename the columns for convinience and easy access of the columns
            df.drop(['time'], axis=1, inplace=True) # Drop the time column, as we are not using it as a time series. We are using the indexes instead
            df.ECG0 = pd.to_numeric(df.ECG0) # The data by default is in the form of an object, Convert each row into numeric or floating point
            df.ECG1 = pd.to_numeric(df.ECG1)
            
            print(filename, len(df))
            
            # Split each file into 6 parts and then make each of them a new row by transposing
            df_split = np.array_split(df, 30) # Split the dataset into 30 different sets. This is not mandatory, but is suggested since the dataset size is less
            for splitted_array in df_split:
                final_list.append(np.array(splitted_array)) # After splitting, we are appending all the splitted arrays into 1 single large array of 3 dimentions
                # The following if-else block is used to create labels. We have taken '1' for AF ECG and '0' for Normal ECG
                # This is not the ideal way to create labels, but this is the most simplest way for this situation
                if cat == 'Normal':
                    labels.append(0)
                if cat == 'Abnormal':
                    labels.append(1)

    # Before returning, convert the lists to arrays and increase the dmentions for being able to feed into the Neural Network
    return np.array(final_list), np.expand_dims(np.array(labels), axis=1)

In [7]:
# Load the training and testing dataset separately by calling the function for each of their root folder locations
X_train, y_train = load_data(root_folder_train)
X_test, y_test = load_data(root_folder_test)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

ValueError: Length mismatch: Expected axis has 1 elements, new values have 3 elements

In [ ]:
# Save the formatted data for easy access later. This is then loaded and used for the neural network
np.save('D:/ECGDATA/Customized_Data/X_train.npy', X_train)
np.save('D:/ECGDATA/Customized_Data/y_train.npy', y_train)
np.save('D:/ECGDATA/Customized_Data/X_test.npy', X_test)
np.save('D:/ECGDATA/Customized_Data/y_test.npy', y_test)